In [124]:
from scipy.optimize import *
import pandas as pd 
import numpy as np 
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from tqdm import tqdm
from pulp import *
import plotly.express as px 
from sklearn.decomposition import PCA
import plotly.io as pio 
pio.renderers.default = "browser"

In [8]:
def meanRetAn(data):             
    Result = 1
    
    for i in data:
        Result *= (1+i)
        
    Result = Result**(1/float(len(data)/52))-1
     
    return(Result)

In [2]:
df = pd.read_csv("WeeklyReturns.csv", index_col="Date")

In [460]:
df

,SPY,IVV,VTI,VOO,QQQ,VEA,IEFA,VWO,VUG,IEMG,...,CHIE,TLEH,RXD,SIJ,SBM,EEH,LTL,HEWW,SCC,SZK
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-05,0.004882,0.004717,0.004218,0.004466,0.013451,-0.005159,-0.005734,0.012149,0.004540,0.011826,...,0.000968,0.016212,-0.001451,0.028376,-0.004653,0.019976,-0.041291,0.000000,0.003489,-0.010947
2016-08-12,0.001283,0.001413,0.001341,0.001499,0.003597,0.019924,0.018791,0.018405,0.002304,0.022027,...,0.000000,0.010117,0.018411,-0.039645,0.010200,0.000000,-0.033489,0.000000,-0.007221,-0.016602
2016-08-19,0.000366,0.000501,0.000893,0.000349,0.000512,-0.000535,-0.001278,0.004191,-0.001327,0.002639,...,0.036750,0.000770,0.007374,-0.007596,-0.013462,0.000000,-0.027237,0.000000,-0.009160,-0.000563
2016-08-26,-0.005720,-0.005732,-0.005351,-0.005784,-0.004093,-0.008032,-0.006948,-0.022692,-0.006286,-0.021277,...,-0.021455,-0.003079,0.036128,0.000998,0.001279,-0.036795,-0.056762,0.000000,0.034258,0.003942
2016-09-02,0.004970,0.004896,0.006635,0.004965,0.002912,0.016734,0.017308,0.016814,0.003296,0.016585,...,-0.001906,0.001158,0.004558,0.007314,-0.005537,0.000000,-0.001616,0.005970,-0.001577,-0.002804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-20,-0.145457,-0.148739,-0.152578,-0.149007,-0.112509,-0.118405,-0.114575,-0.133148,-0.141707,-0.136955,...,-0.116279,-0.121330,0.235641,0.348543,0.117157,0.000000,-0.192487,-0.078797,0.073620,0.301099
2020-03-27,0.107605,0.106736,0.107392,0.104963,0.087843,0.120598,0.125286,0.059494,0.104405,0.064820,...,-0.026316,0.069379,-0.187339,-0.277726,-0.112536,-0.084730,0.110298,-0.018422,-0.174150,-0.227636
2020-04-03,-0.020638,-0.020532,-0.027968,-0.020785,-0.010416,-0.029869,-0.036367,-0.007293,-0.021980,-0.004547,...,0.060489,0.008284,-0.052838,0.089135,0.028774,-0.057986,-0.005603,-0.026945,0.076125,-0.009482


In [210]:
from tqdm import tqdm
from itertools import combinations
samples = []
tickers = df.columns[1:20]
for i in tqdm(range(2,len(tickers)+1,2)):
    samples.extend([list(x) for x in combinations(tickers, i)])

100%|██████████| 9/9 [00:00<00:00, 26.03it/s]


In [212]:
len(samples)

262143

In [205]:
PDI_DF = pd.DataFrame(PDI_dict).T
PDI_DF["Number of Assets"] = PDI_DF["Number of Assets"].astype(int)
PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)

In [208]:
fig = px.scatter(PDI_DF,  x="PDI_INDEX", y="Sharpe Ratio", color="Number of Assets")
fig.show()


In [11]:
tt = [0]
tt.extend([x for x in range(2,11)])

In [12]:
tt

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [4]:
import gurobipy as gp
from gurobipy import GRB

In [24]:
meanRetAn(df["SPY"])

0.09653686994626942

In [158]:
antal = 10
test = df[['JHMT', 'PRNT', 'EUDG', 'JHMH', 'FXZ', 'PEZ']]
print(test.max())
print(test.min())

JHMT    0.171825
PRNT    0.175176
EUDG    0.137244
JHMH    0.167245
FXZ     0.149961
PEZ     0.182065
dtype: float64
JHMT   -0.114077
PRNT   -0.135496
EUDG   -0.133636
JHMH   -0.134068
FXZ    -0.153406
PEZ    -0.218829
dtype: float64


In [159]:
    pca = PCA()
    corr_matrix = np.array(test.corr())
    principalComponents = pca.fit(corr_matrix)
    PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

In [170]:
fundamental_df = pd.read_csv("fund_risk_cluster_reduced.csv",index_col="Ticker") #Fundamental Clustering Data
weekly_return = pd.read_csv("WeeklyReturns.csv",index_col="Date") #Weekly Return Data
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(weekly_return.columns)]

In [172]:
fundamental_df

,Category,Fundamental Cluster,Risk Cluster
SPY,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
IVV,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VTI,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VOO,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
QQQ,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
...,...,...,...
EEH,Large Blend,"Low Fundamental ETF’s (Energy, Health,Finance)",Moderat/High Risk - Slightly Above Market Vola...
LTL,Trading--Leveraged Equity,Market Leveraged ETF,High Risk - Double Market Volatile
HEWW,Miscellaneous Region,Low Fundamentals Broad Market,High Risk - Double Market Volatile
SCC,Trading--Inverse Equity,Market Inverse ETF,Betting against the market


In [193]:
fundamental_df.loc[['VTV', 'CIBR', 'PRN', 'XHE', 'NXTG'],:]


,Category,Fundamental Cluster,Risk Cluster
VTV,Large Value,Mid Fundamentals Broad Market,Moderat/High Risk - Slightly Above Market Vola...
CIBR,Technology,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
PRN,Industrials,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
XHE,Health,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
NXTG,Technology,Mid Fundamentals Broad Market,Moderat/High Risk - Slightly Above Market Vola...


In [194]:
df[['VTV', 'CIBR', 'PRN', 'XHE', 'NXTG']].corr()


,VTV,CIBR,PRN,XHE,NXTG
VTV,1.000000,0.815797,0.906045,0.852176,0.836349
CIBR,0.815797,1.000000,0.860563,0.840134,0.803799
PRN,0.906045,0.860563,1.000000,0.862444,0.838910
XHE,0.852176,0.840134,0.862444,1.000000,0.800139
NXTG,0.836349,0.803799,0.838910,0.800139,1.000000


In [195]:
fundamental_df.loc[['SCHD', 'MGK', 'BBH', 'SCHG', 'RDVY'],:]

,Category,Fundamental Cluster,Risk Cluster
SCHD,Large Value,Mid Fundamentals Broad Market,Moderat/High Risk - Slightly Above Market Vola...
MGK,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
BBH,Health,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
SCHG,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
RDVY,Large Value,Mid Fundamentals Broad Market,Moderat/High Risk - Slightly Above Market Vola...


In [196]:
df[['SCHD', 'MGK', 'BBH', 'SCHG', 'RDVY']].corr()

,SCHD,MGK,BBH,SCHG,RDVY
SCHD,1.000000,0.931716,0.689906,0.942048,0.945722
MGK,0.931716,1.000000,0.754849,0.994882,0.911086
BBH,0.689906,0.754849,1.000000,0.759271,0.699064
SCHG,0.942048,0.994882,0.759271,1.000000,0.933393
RDVY,0.945722,0.911086,0.699064,0.933393,1.000000


In [183]:
print(np.round(principalComponents.explained_variance_ratio_,10))

[0.32885906 0.30258991 0.15747073 0.1295195  0.0815608  0.        ]


In [186]:
PDI

3.6646661098491924

In [185]:
sum(principalComponents.explained_variance_ratio_[:5])

1.0000000000000002

In [177]:
print(len(principalComponents.explained_variance_ratio_))
np.round(principalComponents.explained_variance_ratio_,4)


6


array([0.3289, 0.3026, 0.1575, 0.1295, 0.0816, 0.    ])

In [168]:
for i in range(1,len(principalComponents.explained_variance_ratio_)+1):
    print(i)

1
2
3
4
5
6


In [165]:
principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1)

array([3.28859063e-01, 6.05179820e-01, 4.72412203e-01, 5.18077981e-01,
       4.07803989e-01, 1.69154207e-31])

In [167]:
df[['FXZ', 'DBEM', 'IEMG', 'IYM', 'PSCI']].corr()

,FXZ,DBEM,IEMG,IYM,PSCI
FXZ,1.000000,0.807496,0.780125,0.964537,0.914688
DBEM,0.807496,1.000000,0.956624,0.801668,0.700336
IEMG,0.780125,0.956624,1.000000,0.774018,0.664428
IYM,0.964537,0.801668,0.774018,1.000000,0.874296
PSCI,0.914688,0.700336,0.664428,0.874296,1.000000


In [163]:
test.corr()

,JHMT,PRNT,EUDG,JHMH,FXZ,PEZ
JHMT,1.000000,0.851209,0.819518,0.838515,0.848346,0.864780
PRNT,0.851209,1.000000,0.775950,0.773711,0.812731,0.797426
EUDG,0.819518,0.775950,1.000000,0.775002,0.843716,0.835978
JHMH,0.838515,0.773711,0.775002,1.000000,0.788983,0.824495
FXZ,0.848346,0.812731,0.843716,0.788983,1.000000,0.833706
PEZ,0.864780,0.797426,0.835978,0.824495,0.833706,1.000000


In [107]:
min = np.min(meanRetAn(test.values))
max = np.max(meanRetAn(test.values))
print(min)
print(max)

-0.08218958696268619
0.2279823374729988


In [108]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(test.apply(meanRetAn) * weights)
    vol = np.sqrt(np.dot(weights.T, np.dot(test.cov()*52, weights)))
    sr = ret/vol
    return np.array([ret, vol, sr])

def neg_sharpe(weights):
# the number 2 is the sharpe ratio index from the get_ret_vol_sr
    return -(get_ret_vol_sr(weights)[2])

def check_sum(weights):
    #return 0 if sum of the weights is 1
    return np.sum(weights)-1

In [109]:
from itertools import repeat

In [110]:
frontier = np.linspace(min,max, 200)
ret, vol, sr, weight = [],[],[],[]
for i in tqdm(frontier):
    cons = ({'type':'eq', 'fun':check_sum},
            {'type' : 'eq', 'fun': lambda w: get_ret_vol_sr(w)[0] - i})
    bounds = [x for x in repeat((0,1),antal )]
    init_guess = [x for x in repeat(0.25, antal)]
    result = minimize(neg_sharpe,init_guess,method='SLSQP', bounds=bounds, constraints=cons)
    ret.append(get_ret_vol_sr(result.x)[0])
    vol.append(get_ret_vol_sr(result.x)[1])
    sr.append(get_ret_vol_sr(result.x)[2])
    weight.append(result.x)





100%|██████████| 200/200 [01:18<00:00,  2.55it/s]


In [114]:
for i in np.round(weight,2):
    print(i)

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.   0.   0.   0.99 0.   0.   0.   0.   0.   0.  ]
[0.   0.   0.   0.99 0.   0.   0.   0.   0.   0.01]
[0.   0.   0.01 0.98 0.   0.   0.   0.   0.   0.01]
[0.   0.   0.01 0.98 0.   0.   0.   0.   0.   0.01]
[0.   0.   0.01 0.97 0.   0.   0.   0.   0.   0.01]
[0.   0.   0.01 0.97 0.   0.   0.   0.   0.   0.02]
[0.   0.   0.02 0.96 0.   0.   0.   0.   0.   0.02]
[0.   0.   0.02 0.96 0.   0.   0.   0.   0.   0.02]
[0.   0.   0.02 0.95 0.   0.   0.   0.   0.   0.03]
[0.   0.   0.02 0.95 0.   0.   0.   0.   0.   0.03]
[0.   0.   0.01 0.94 0.   0.   0.   0.   0.   0.05]
[0.   0.   0.   0.62 0.   0.   0.38 0.   0.   0.  ]
[0.   0.   0.   0.59 0.   0.   0.41 0.   0.   0.  ]
[0.   0.   0.   0.56 0.   0.   0.44 0.   0.   0.  ]
[0.   0.   0.   0.53 0.   0.   0.47 0.   0.   0.  ]
[0.  0.  0.  0.5 0.  0.  0.5 0.  0.  0. ]
[0.   0.   0.   0.47 0.   0.   0.53 0.   0.   0.  ]
[0.   0.   0.   0.43 0.   0.   0.57 0.   0.   0.  ]
[0.  0.  0.  0.4 0.  0.  0.6 0.  0.  0. ]


In [115]:

ef_df = pd.DataFrame( columns = ["Return", "Volatilty","Sharpe Ratio","Weights"])
ef_df["Return"] = ret 
ef_df["Volatilty"] = vol
ef_df["Sharpe Ratio"] = sr
ef_df["Weights"] = list(np.round(weight,3))


In [116]:
fig = px.scatter(ef_df,  x="Volatilty", y="Return", color="Sharpe Ratio")
fig.show()

In [101]:
print(result)

     fun: -0.7095294264135118
     jac: array([ 0.35250189,  0.52085974,  0.47955945,  2.05730546,  0.26351862,
        0.11878253,  0.14666863,  0.09085836,  0.23814747,  0.21499847,
        0.05600017,  0.42456649,  0.3483988 ,  0.24385005,  0.77278922,
       -0.01458228,  0.71973389,  0.47195834, -0.02258866,  0.37616308,
        0.80027892,  0.05558895,  0.62710626,  0.14859862,  0.40420367,
        0.14743898,  0.13611188,  0.22413629,  0.26986694,  0.13495099,
        0.46998543, -0.02018506,  0.15646839,  2.47667145,  0.04139415,
        0.22065767,  0.0821027 ,  0.17949991,  0.40921518,  0.60552275,
        0.36650638,  0.25017932,  0.55509944, -0.04178225, -0.        ,
       -0.07800882,  0.49502426,  0.29358963,  0.74012269,  0.78089415])
 message: 'Optimization terminated successfully'
    nfev: 153
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([2.58907479e-15, 0.00000000e+00, 0.00000000e+00, 2.11061637e-15,
       0.00000000e+00, 3.16012407e-15, 4.665

In [23]:
get_ret_vol_sr(result.x)

array([0.23338693, 0.21881078, 1.06661528])

In [27]:
np.sum(result.x)

1.0000000000001832

In [32]:
float(2.8235524*10**(-15))

2.8235524000000003e-15

In [489]:
test = df.iloc[:,:50]
assets = list(test.columns)
returns = dict(test.apply(meanRetAn))
assets_load = LpVariable.dicts("Loadings",assets,lowBound=0,cat='Continuous')

In [490]:
prob = LpProblem("Simple Diet Problem",LpMaximize)


In [498]:

prob += lpSum([returns[i]*assets_load[i] for i in assets]) / np.sqrt(lpDot(assets_load.T, lpDot(test.cov()*52, weights)))
prob += lpSum([assets_load[i] for i in assets]) == 1 
prob += np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252, weights)))



AttributeError: 'dict' object has no attribute 'T'

In [496]:
prob.solve()
print("Status:", LpStatus[prob.status])


Status: Optimal


In [497]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

Loadings_ARKK = 1.0
Loadings_DIA = 0.0
Loadings_EEM = 0.0
Loadings_EFA = 0.0
Loadings_GDX = 0.0
Loadings_IEFA = 0.0
Loadings_IEMG = 0.0
Loadings_IJH = 0.0
Loadings_IJR = 0.0
Loadings_ITOT = 0.0
Loadings_IVE = 0.0
Loadings_IVV = 0.0
Loadings_IVW = 0.0
Loadings_IWB = 0.0
Loadings_IWD = 0.0
Loadings_IWF = 0.0
Loadings_IWM = 0.0
Loadings_IWR = 0.0
Loadings_IXUS = 0.0
Loadings_MDY = 0.0
Loadings_QQQ = 0.0
Loadings_QUAL = 0.0
Loadings_RSP = 0.0
Loadings_SCHB = 0.0
Loadings_SCHD = 0.0
Loadings_SCHF = 0.0
Loadings_SCHX = 0.0
Loadings_SDY = 0.0
Loadings_SPY = 0.0
Loadings_USMV = 0.0
Loadings_VB = 0.0
Loadings_VBR = 0.0
Loadings_VEA = 0.0
Loadings_VEU = 0.0
Loadings_VGT = 0.0
Loadings_VIG = 0.0
Loadings_VO = 0.0
Loadings_VOO = 0.0
Loadings_VT = 0.0
Loadings_VTI = 0.0
Loadings_VTV = 0.0
Loadings_VUG = 0.0
Loadings_VV = 0.0
Loadings_VWO = 0.0
Loadings_VXUS = 0.0
Loadings_VYM = 0.0
Loadings_XLF = 0.0
Loadings_XLK = 0.0
Loadings_XLV = 0.0
Loadings_XLY = 0.0


In [485]:
dict(df[["SPY","QQQ"]].apply(meanRetAn))

{'SPY': 0.09653686994626942, 'QQQ': 0.19409982256452252}

In [406]:
import gurobipy as gp
from gurobipy import GRB

In [407]:
# Resource and job sets

#assets = list(df.columns)
assets , returns = gp.multidict(df.apply(meanRetAn))

In [408]:
assets

['SPY', 'IVV', 'VTI', 'VOO', 'QQQ', 'VEA', 'IEFA', 'VWO', 'VUG', 'IEMG']

In [409]:
# Declare and initialize model
m = gp.Model('lp')

In [410]:
# Create decision variables for the RAP model
x = m.addVars(assets, name="assign", lb=0.0, vtype=GRB.CONTINUOUS)

In [416]:
# Create job constraints
assets = m.addConstrs((x.sum(j,'*') == 1 for j in assets) , name='assets')


In [417]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(returns), GRB.MAXIMIZE)

In [418]:
# Save model for inspection
m.write('lp.lp')

In [419]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 20 rows, 10 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.7566296e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  7.756629648e-01


In [420]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[SPY] 1.0
assign[IVV] 1.0
assign[VTI] 1.0
assign[VOO] 1.0
assign[QQQ] 1.0
assign[VEA] 1.0
assign[IEFA] 1.0
assign[VWO] 1.0
assign[VUG] 1.0
assign[IEMG] 1.0
Total matching score:  0.7756629648282669


In [22]:
port_weekly_return = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)

NameError: name 'weekly_returns' is not defined

In [21]:
list(df[["SPY","QQQ"]])

['SPY', 'QQQ']

In [3]:
# Declare and initialize model
m = gp.Model('RAP')

Academic license - for non-commercial use only - expires 2021-06-04
Using license file /Users/rasmusblirupjensen/gurobi.lic


In [2]:
# Create decision variables for the RAP model
x = m.addVars(combinations, name="assign")

NameError: name 'm' is not defined